In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer2 import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import cdd

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import time
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [4]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")

box_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'right_sweeper'))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'left_sweeper'))




locs = [[0.,0.,0.],
        [0,1,0.85],
        [0,-1,0.55]]
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))

t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
                 t1)
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 t2)


plant.Finalize()

idx = 0
q0 = [0.0, 0.0]
# q_low  = [-1.7, -1.7, 0]
# q_high = [1.7, 1.7,  0]
eps = 0# 1e-3
val = 1.7
q_low  = [-val+eps, -val+eps, 0]
q_high = [val-eps, val-eps,  0]
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(3)

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), q_star)[:2])

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

# This line will run marching cubes to generate a mesh of the C-space obstacle
# Increase N to increase the resolution of the C-space obstacle.


INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Meshcat listening for connections at http://localhost:7001


In [5]:
visualizer.visualize_collision_constraint2d(factor = 1.2, num_points = 100)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
visualizer.meshcat2.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))


## Set up the sliders so we can move the plant around manually

In [7]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=1.7, min=-1.7)

In [8]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
right_sweeper_gids = [gid for gid in gids if "right_sweeper::" in get_name_of_gid(gid)]
left_sweeper_gids = [gid for gid in gids if "left_sweeper::" in get_name_of_gid(gid)]

right_sweeper_fused_col_geom = right_sweeper_gids[2:]
right_sweeper_fused_set = GeometrySet(right_sweeper_fused_col_geom)
left_sweeper_fused_col_geom = left_sweeper_gids[4:]
left_sweeper_fused_set = GeometrySet(left_sweeper_fused_col_geom)
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(right_sweeper_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(left_sweeper_fused_set))

right_sweeper_end_gid = right_sweeper_gids[-1]
left_sweeper_end_gid = left_sweeper_gids[-1]
id_pairs_of_interest = [(right_sweeper_end_gid, left_sweeper_end_gid),
                       ]
visualizer.collision_pairs_of_interest = id_pairs_of_interest


# Setup IRIS Options and Generate Regions

In [9]:
# Some examples of some seed points which give large regions using the
# non-linear programming approach to IRIS described in Algorithm 3
seed_points_q = np.array([   [0.0, 0],
                              [0.7, -0.9],
                              [-0.5, -0.5],
                              [0.4,-1.3]
                              ])
seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((2,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)


In [10]:
regions = []
ellipses = []



iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-5
iris_options.max_faces_per_collision_pair = 60
iris_options.termination_threshold = -1
iris_options.q_star = np.zeros(2)
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False

def promote_region_to_3d(region, width = 0.2):
    A = block_diag(region.A(), np.array([-1,1])[:, np.newaxis])
    b = np.append(region.b(), width*np.ones(2))

    return HPolyhedron(A,b)
    
regions_3d = []

for i, s in enumerate(seed_points):
    q = Ratfk.ComputeQValue(s, np.zeros((2,)))
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), q)
    r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    regions.append(r)
    print(r.IsBounded())
    regions_3d.append(promote_region_to_3d(r))
    ellipses.append(r.MaximumVolumeInscribedEllipsoid())
    
# chosen for colorblind compatability
colors_dict = {
    0: (144,144,144), #grayish
    1:(30,136,229), # bluish
    2: (255, 193, 7), # gold
    3: (0, 140, 6), # green    
}
if do_viz:
    visualizer.plot_regions(regions,
                            ellipses=ellipses,
                            region_suffix='_original',
                            colors = list(colors_dict.values()),
                            wireframe = False,
                           opacity = 0.2)

True
True
True
True


In [11]:
r.IsBounded()

True

# Certify regions

In [12]:
cspace_free_region_certifier = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
filtered_collision_pairs = set()
solver_options = mp.SolverOptions()
# make the solver verbose
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 0)

## first search for a minimal, uniform contraction of a proposed region which can be certified

In [13]:
# i = 0
# bisect_regions = []
# bisect_solutions = []
# for i in range(len(regions)):
#     print(i)
#     region_to_certify = regions[i]
#     seed_point = seed_points[i,:]


#     binary_search_options = rational_forward_kinematics.BinarySearchOption()
#     binary_search_options.epsilon_max = 0 # it is very unlikely that we can find a uniform expansion of the current region
#     binary_search_options.max_iters = 15
#     binary_search_options.gap_tol = 1e-4
#     # speed up the bisection search by taking non-uniform steps when possible
#     binary_search_options.search_d = True
#     # find the smallest e such that At <= b + e1 still contains our seed point.
#     binary_search_options.epsilon_min = rational_forward_kinematics.FindEpsilonLower(region_to_certify.A(),
#                                                                                      region_to_certify.b(),
#                                                                                      limits_t[0], limits_t[1],
#                                                                                      seed_point)
#     #use as many threads as possible to speed up computation
# #     binary_search_options.num_threads = 1

#     certified_region_contraction_solution = cspace_free_region_certifier.CspacePolytopeBinarySearch(
#                                                                      q_star[:2],
#                                                                      filtered_collision_pairs,
#                                                                      region_to_certify.A(),
#                                                                      region_to_certify.b(),
#                                                                      binary_search_options, 
#                                                                      solver_options,
#                                                                      seed_point)
#     bisect_solutions.append(certified_region_contraction_solution)
#     certified_region_contraction = HPolyhedron(certified_region_contraction_solution.C,
#                                                certified_region_contraction_solution.d)
    
#     bisect_regions.append(certified_region_contraction)
# if do_viz:
#     visualizer.plot_regions(bisect_regions, ellipses=None,
#                             region_suffix='_certified_region_contraction',
#                             wireframe = False)
# visualizer.certified_region_solution_list = bisect_solutions

In [14]:
regions

In [39]:
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra
from pydrake.all import set_log_level
from pydrake.all import logging
log = logging.getLogger("drake")
log.setLevel("DEBUG")

start_point = Ratfk.ComputeTValue(np.array([0,0.]), q_star)
# end_point = np.array([-1.1,0])
end_point = Ratfk.ComputeTValue(np.array([0,0.5]), q_star)

spp = BsplineTrajectoryThroughUnionOfHPolyhedra(start_point, end_point, [regions[0]])
spp.set_max_velocity([.8, .8])
spp.set_extra_control_points_per_region(10)

In [40]:

traj, result = spp.Solve()


In [41]:
regions[0].PointInSet(end_point)

True

In [36]:
regions[0].A()@end_point<= regions[0].b()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False, False,
        True,  True,  True])

In [18]:
self = visualizer
substeps = 1000
runtime = 5*substeps
def animate_s(traj, step, runtime):
    # loop
    idx = 0
    going_fwd = True
    time_points = np.linspace(0, traj.end_time(), steps)
    
    for _ in range(runtime):
        # print(idx)
        q = self.forward_kin.ComputeQValue(traj.value(time_points[idx]), self.q_star)
        self.showres(q)
        if going_fwd:
            if idx + 1 < steps:
                idx += 1
            else:
                going_fwd = False
                idx -= 1
        else:
            if idx - 1 >= 0:
                idx -= 1
            else:
                going_fwd = True
                idx += 1
animate_s(traj, substeps, runtime )

AttributeError: 'NoneType' object has no attribute 'end_time'